In [195]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Based on my research for Kmean memory leak if OMP_NUM_THREADS=1 is set in enironment variable it would slow down the process as it limits to one thread and suppressing the warning is recommended to over come the slowness
import warnings
warnings.filterwarnings('ignore')

In [196]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [197]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [198]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [199]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
column_names = df_market_data.columns.values.tolist()
# df_market_data_normalized = StandardScaler().fit_transform(df_market_data[column_names])
df_market_data_normalized = StandardScaler().fit_transform(df_market_data)

In [200]:
# create a reuable function to copy crypto names and set index

def copy_cryptoname_setindex(df_orig,column_headers,df_with_data):

    # Create a DataFrame
    df_with_index = pd.DataFrame(df_with_data, columns=column_headers)

    # Copy the crypto names from the original data
    df_with_index['coin_id']=df_orig.index

    # Set the coinid column as index
    df_with_index=df_with_index.set_index('coin_id')

    return df_with_index

In [201]:
# # Create a DataFrame with the scaled data
# # Copy the crypto names from the original data
# # Set the coinid column as index
# calling the resuable function
df_market_scaled = df_market_data.pipe(copy_cryptoname_setindex, column_names,df_market_data_normalized)

# Display sample data
df_market_scaled.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


---

### Find the Best Value for k Using the Original Data.

In [202]:
# reusable function to create a list with range of values
def create_list_with_range_value(start_value, end_value):
    return list(range(int(start_value),int(end_value)))


In [203]:
# Create a list with the number of k-values from 1 to 11
kvalues=create_list_with_range_value(1,11)
kvalues

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [204]:
# resuable function to get inertia value
def find_inertia_values(input_df, input_range):
    # Create an empty list to store the inertia values
    return_list = []
    # Inside the loop:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    # 2. Fit the model to the data using `df_market_data_scaled`
    # 3. Append the model.inertia_ to the inertia list
    for each_value in input_range: 
        model = KMeans(n_clusters=each_value,random_state=0)
        model.fit(input_df)
        return_list.append(model.inertia_)
    # return the inertia value
    return return_list

In [205]:
# # Create an empty list to store the inertia values
# # Create a for loop to compute the inertia with each possible value of k
# # Inside the loop:
# # 1. Create a KMeans model using the loop counter for the n_clusters
# # 2. Fit the model to the data using `df_market_data_scaled`
# # 3. Append the model.inertia_ to the inertia list
# calling reusable function to get inertia value
inertia_values = find_inertia_values(df_market_scaled,kvalues)

inertia_values

[287.0,
 195.82021818036043,
 123.19048183836958,
 79.02243535120977,
 65.302379141625,
 52.888518219775314,
 43.91469044757747,
 37.517032490103574,
 32.485240832183536,
 28.22289929060893]

In [206]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k': kvalues,
    'Inertia': inertia_values,
}

# Create a DataFrame with the data to plot the Elbow curve
df_for_elbow_curve = pd.DataFrame(elbow_data)


In [207]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_for_Kvalue = df_for_elbow_curve.hvplot.line(
    x='k', 
    y='Inertia', 
    title='Elbow Curve for K Value', 
    frame_width=400, 
    frame_height=200)

elbow_plot_for_Kvalue

:Curve   [k]   (Inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [208]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4,random_state=0)

In [209]:
# Fit the K-Means model using the scaled data
model.fit(df_market_scaled)

KMeans(n_clusters=4, random_state=0)

In [210]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_prediction = model.predict(df_market_scaled)

# Print the resulting array of cluster values.
print(cluster_prediction)

[0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


In [211]:
# Create a copy of the DataFrame
df_market_scaled_copy = df_market_scaled.copy()


In [212]:
# Add a new column to the DataFrame with the predicted clusters
df_market_scaled_copy['clusterprediction'] = cluster_prediction

# Display sample data
df_market_scaled_copy.head(5)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,clusterprediction
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,3
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,3
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [213]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_scaled_plot = df_market_scaled_copy.hvplot.scatter(
    title='Crypto Currency Segment',
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='clusterprediction',
    hover_cols=['coin_id'],
    frame_width=400,
    frame_height=200
)
market_scaled_plot

:NdOverlay   [clusterprediction]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [214]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [215]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
fit_transform_data = pca_model.fit_transform(df_market_scaled)
# View the first five rows of the DataFrame. 
fit_transform_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [216]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_variance = pca_model.explained_variance_ratio_

print(f'PC1 has {pca_variance[0]*100:.2f}%')
print(f'PC2 has {pca_variance[1]*100:.2f}%')
print(f'PC3 has {pca_variance[2]*100:.2f}%')
(print(f'Sum of total explained variance of the three principal components is {sum(pca_variance)*100:.2f}%'))

PC1 has 37.20%
PC2 has 34.70%
PC3 has 17.60%
Sum of total explained variance of the three principal components is 89.50%


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.50%

In [217]:
# set the column names
column_titles = ['PC1' , 'PC2' , 'PC3']

# # Create a new DataFrame with the PCA data.
# # Copy the crypto names from the original data
# # Set the coinid column as index
# calling the resuable function
df_new_market_pca_data = df_market_scaled.pipe(copy_cryptoname_setindex, column_titles,fit_transform_data)

# Display sample data
df_new_market_pca_data.head(10)


,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715
binancecoin,-0.516534,1.388377,0.804071
chainlink,-0.450711,0.517699,2.846143
cardano,-0.345600,0.729439,1.478013
litecoin,-0.649468,0.432165,0.600303


---

### Find the Best Value for k Using the PCA Data

In [218]:
# Create a list with the number of k-values from 1 to 11
k_values_list=create_list_with_range_value(1,11)
k_values_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [219]:
# # Create an empty list to store the inertia values
# # Create a for loop to compute the inertia with each possible value of k
# # Inside the loop:
# # 1. Create a KMeans model using the loop counter for the n_clusters
# # 2. Fit the model to the data using `df_market_data_pca`
# # 3. Append the model.inertia_ to the inertia list
# calling reusable function to get inertia value
inertia_list = find_inertia_values(df_new_market_pca_data,k_values_list)

inertia_list

[256.87408556789234,
 165.90199402036006,
 93.77462568057298,
 49.66549665179737,
 37.878747033462474,
 27.618971787957435,
 21.182775862957328,
 17.389823204768888,
 13.593650379876722,
 10.559357562793421]

In [220]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve_dict ={'k_values_list' : k_values_list , 'inertia_list' : inertia_list}

# Create a DataFrame with the data to plot the Elbow curve
df_for_elbow_curve = pd.DataFrame(elbow_curve_dict)
df_for_elbow_curve.head()

,k_values_list,inertia_list
0,1,256.874086
1,2,165.901994
2,3,93.774626
3,4,49.665497
4,5,37.878747


In [221]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_elbow_curve = df_for_elbow_curve.hvplot.line(
    x="k_values_list",
    y="inertia_list",
    title="Elbow Curve to Visually identify the Optimal value for K",
    xticks = k_values_list,
    frame_width=400,
    frame_height=200

)
elbow_plot_elbow_curve


:Curve   [k_values_list]   (inertia_list)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No.Still visually it appears the best value for K seems to be 4

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [222]:
# Initialize the K-Means model using the best value for k
pca_kmean_model = KMeans(n_clusters=4, random_state=0)


In [223]:
# Fit the K-Means model using the PCA data
pca_kmean_model.fit(df_new_market_pca_data)

KMeans(n_clusters=4, random_state=0)

In [224]:
# Predict the clusters to group the cryptocurrencies using the PCA data
cluster_group = pca_kmean_model.predict(df_new_market_pca_data)

# Print the resulting array of cluster values.
print(cluster_group)

[0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


In [225]:
# Create a copy of the DataFrame with the PCA data
df_new_market_pca_data_copy = df_new_market_pca_data.copy()

# Add a new column to the DataFrame with the predicted clusters
df_new_market_pca_data_copy['Cluster_Group'] = cluster_group

# Display sample data
df_new_market_pca_data_copy.head()

,PC1,PC2,PC3,Cluster_Group
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,0
ethereum,-0.458261,0.458466,0.952877,0
tether,-0.433070,-0.168126,-0.641752,3
ripple,-0.471835,-0.222660,-0.479053,3
bitcoin-cash,-1.157800,2.041209,1.859715,0


In [226]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
new_market_pca_plot = df_new_market_pca_data_copy.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='Cluster_Group',
    xlabel='PC1',
    ylabel='PC2',
    hover_cols=['coin_id'],
    frame_width=400,
    frame_height=200
)
new_market_pca_plot

:NdOverlay   [Cluster_Group]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [227]:
# Composite plot to contrast the Elbow curves
elbow_plot_for_Kvalue + elbow_plot_elbow_curve

:Layout
   .Curve.I  :Curve   [k]   (Inertia)
   .Curve.II :Curve   [k_values_list]   (inertia_list)

In [228]:
# Composite plot to contrast the clusters
market_scaled_plot + new_market_pca_plot

:Layout
   .NdOverlay.I  :NdOverlay   [clusterprediction]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Cluster_Group]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Even though the optimal k value with both original data and PCA data seems to be 4, the elbow curve plot shows that the elbow curve with PCA data is lower when compared to original data. cluster scatter plot visually shows that the clustered scatter plot with PCA data is much closer or in other words it is tightly packed and clearly differentiates the outliers when compared to the cluster scatter plot with original data.